# Introduction
Use RBM to perform feature extraction on an image-based dataset that you find or create. If you go this route, present the features you extract and explain why this is a useful feature extraction method in the context you’re operating in. DO NOT USE either the MNIST digit recognition database or the iris data set. They’ve been worked on in very public ways very very many times and the code is easily available. (However, that code could be a useful resource to refer to).

## Import the Data and Cleaning

https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
import random
from sklearn.neural_network import BernoulliRBM
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
import seaborn as sns
import time
from sklearn.metrics import confusion_matrix

%matplotlib inline

In [2]:
def load_lungs(data_path, categories, img_size):
    """Build dataset of x-rays
    Parameters:
    data_path(str): file location
    categories(str): folders within file location
    img_size(int): dimensions of image
    
    Returns:
    Tuple of the data and targets"""
    
    data = []
    
    for category in categories:
        #path to images
        path = os.path.join(data_path, category)
        # image identifer 0 for normal, 1 for pneumonia
        class_num = categories.index(category)
        
        for img in os.listdir(path):    
            # append each image to array apply grayscale
            img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
            
            # uniform image shape for all x-rays
            new_array = cv2.resize(img_array, (img_size, img_size))
           
            data.append([new_array, class_num])
    
    # randomize the data to reduce bias
    random.shuffle(data)
    
    return data

In [3]:
def feature_target(dataset):
    """Append features and target variables
    Parameters:
    dataset(array): dataset that will be split
    
    Returns: X and y variables for modeling
    """
    X = []
    y = []

    for features, label in dataset:
        X.append(features)
        y.append(label)
    
    X = np.array(X)
    X = X.reshape(X.shape[0], -1)
    #X = np.array(X).reshape(-1, img_size, img_size, 1)
    return X, y

In [4]:
start_time = time.time()
data_path = 'pneumonia_data/test'
categories = ['NORMAL', 'PNEUMONIA']
img_size = 1500

train_set = load_lungs(data_path, categories, img_size)
print('Loaded {} x-rays.'.format(len(train_set)))

X_train, y_train = feature_target(train_set)
print("\n--- %s seconds ---" % (time.time() - start_time))

Loaded 624 x-rays.

--- 6.761709690093994 seconds ---


In [5]:
start_time = time.time()
data_path = 'pneumonia_data/val'

test_set = load_lungs(data_path, categories, img_size)
print('Loaded {} x-rays.'.format(len(test_set)))

X_test, y_test = feature_target(test_set)
print("\n--- %s seconds ---" % (time.time() - start_time))

Loaded 624 x-rays.

--- 0.20566725730895996 seconds ---


In [ ]:
start_time = time.time()

rbm = BernoulliRBM()
rbm = rbm.fit(X_train, y_train)
print('Training Set Fitted')

pred_train = rbm.predict(X_train)
pred_test = rbm.predict(X_test)
print('Test Set Predicted')

# Print results of test and train data
print('Cross Validation of Training Set:')
print(cross_val_score(neighbors, X_train, y_train, cv=5))
print('\nCross Validation of Test Set:')
print(cross_val_score(neighbors, X_test, y_test, cv=5))

print('\nAccuracy Score of Training Set: {}\n'.format(accuracy_score(y_train, pred_train)))
print('Accuracy Score of Test Set: {}'.format(accuracy_score(y_test, pred_test)))
print("\n--- %s seconds ---" % (time.time() - start_time))  

In [ ]:
# create heatmap of confusion matrix
plt.figure(figsize=(15, 15))
plt.subplot(2, 1, 1)
sns.heatmap(confusion_matrix(y_train, pred_train), annot=True, fmt='g')

plt.ylabel('True Label', fontweight='bold')
plt.xlabel('Predicted Label', fontweight='bold')
plt.title('Confusion Matrix of Training Set', fontweight='bold')

plt.subplot(2, 1, 2)
sns.heatmap(confusion_matrix(y_test, pred_test), annot=True, fmt='g')

plt.ylabel('True Label', fontweight='bold')
plt.xlabel('Predicted Label', fontweight='bold')
plt.title('Confusion Matrix of Test Set', fontweight='bold')

plt.tight_layout()
plt.show()